In [1]:
import numpy as np
import pandas as pd
import datetime

In [2]:
exports = pd.read_csv("Export1.txt.gz", sep=";", decimal=',');
exports.columns = "Flux Mois Annee Code_NC8 Country EUROS KG USUP".split()



In [3]:
exports.head()

,Flux,Mois,Annee,Code_NC8,Country,EUROS,KG,USUP
0,E,10,2012,10011100,CH,787.0,981.0,0.0
1,E,10,2012,10011100,DE,26476.0,19252.0,0.0
2,E,10,2012,10011100,ES,278241.0,1037556.0,0.0
3,E,10,2012,10011100,GB,665005.0,2104250.0,0.0
4,E,10,2012,10011100,GR,6470.0,9970.0,0.0


In [4]:
# Not sure of this .... need a real wine expert !!!
wine_mask = (exports['Code_NC8'] >= 22040000);
wine_mask = wine_mask & (exports['Code_NC8'] <= 22049999);
wine_exports = exports[wine_mask].copy();
wine_exports['Code_NC8'] = wine_exports['Code_NC8'].apply(str)

In [5]:
wine_exports['Month'] = wine_exports.Annee.apply(str) + "-" + wine_exports.Mois.apply(str) + "-01";
wine_exports['Month'] = wine_exports['Month'].apply(lambda x : datetime.datetime.strptime(x, "%Y-%m-%d"))

In [6]:
wine_exports.head()

,Flux,Mois,Annee,Code_NC8,Country,EUROS,KG,USUP,Month
18798,E,10,2012,22041011,AD,229331.0,11380.0,9384.0,2012-10-01
18799,E,10,2012,22041011,AE,2420711.0,94908.0,83784.0,2012-10-01
18800,E,10,2012,22041011,AF,13950.0,1380.0,1800.0,2012-10-01
18801,E,10,2012,22041011,AG,18198.0,872.0,872.0,2012-10-01
18802,E,10,2012,22041011,AL,557.0,40.0,36.0,2012-10-01


In [7]:
wine_exports.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 149190 entries, 18798 to 10011999
Data columns (total 9 columns):
Flux        149190 non-null object
Mois        149190 non-null int64
Annee       149190 non-null int64
Code_NC8    149190 non-null object
Country     149183 non-null object
EUROS       149190 non-null float64
KG          149190 non-null float64
USUP        149190 non-null float64
Month       149190 non-null datetime64[ns]
dtypes: datetime64[ns](1), float64(3), int64(2), object(3)
memory usage: 11.4+ MB


In [8]:
wine_exports.describe()

,Mois,Annee,EUROS,KG,USUP
count,149190.000000,149190.000000,1.491900e+05,1.491900e+05,1.491900e+05
mean,6.567538,2013.775474,2.207440e+05,4.172200e+04,4.032514e+04
std,3.479498,1.400818,1.575743e+06,2.111715e+05,2.030953e+05
min,1.000000,2012.000000,1.000000e+00,0.000000e+00,1.000000e+00
25%,4.000000,2013.000000,1.239000e+03,2.250000e+02,2.110000e+02
50%,7.000000,2014.000000,6.561000e+03,1.260000e+03,1.200000e+03
75%,10.000000,2015.000000,4.235675e+04,9.273750e+03,9.023750e+03
max,12.000000,2016.000000,8.537028e+07,6.678057e+06,6.670051e+06


In [9]:
labels1 = pd.read_csv("codes.txt.gz", sep=";", decimal=',');
labels1.columns = "Code_NC8 Label".split()
labels1['Code_NC8'] = labels1['Code_NC8'].apply(str)

wine_exports2 = wine_exports.merge(labels1 , how='left', on='Code_NC8');


def normalize_wine_name_and_variant(full_name):
    reduced = full_name.split(",")[0];
    lFrench = True;
    if("non produits dans l'UE" in full_name):
        reduced = "other";
        lFrench = False;
    elif("produits dans l'UE" in full_name):
        reduced = "europe";
        lFrench = False;
    Kept = ['champagne', 'bordeaux', 'bourgogne', 'alsace', 'rhône' , 'languedoc', 'beaujolais', 'loire' , 'europe']
    lWine = "other";
    if(lFrench):
        lWine = "france";
    for wine in Kept:
        if(wine in reduced.lower()):
            lWine = wine;
    Variants=['blanc' , 'mousseux']
    lVariant = "rouge";
    # default value for champagne is not 'rouge'
    if(lWine == "champagne"):
        lVariant = "mousseux";
    for var in Variants:
        if(var in reduced.lower()):
            lVariant = var;
    return (lWine.upper(), lVariant.upper());
    
labels1.head();

In [10]:
wine_exports.head()

,Flux,Mois,Annee,Code_NC8,Country,EUROS,KG,USUP,Month
18798,E,10,2012,22041011,AD,229331.0,11380.0,9384.0,2012-10-01
18799,E,10,2012,22041011,AE,2420711.0,94908.0,83784.0,2012-10-01
18800,E,10,2012,22041011,AF,13950.0,1380.0,1800.0,2012-10-01
18801,E,10,2012,22041011,AG,18198.0,872.0,872.0,2012-10-01
18802,E,10,2012,22041011,AL,557.0,40.0,36.0,2012-10-01


In [11]:
wine_exports2.head()

,Flux,Mois,Annee,Code_NC8,Country,EUROS,KG,USUP,Month,Label
0,E,10,2012,22041011,AD,229331.0,11380.0,9384.0,2012-10-01,"Champagne, avec AOP"
1,E,10,2012,22041011,AE,2420711.0,94908.0,83784.0,2012-10-01,"Champagne, avec AOP"
2,E,10,2012,22041011,AF,13950.0,1380.0,1800.0,2012-10-01,"Champagne, avec AOP"
3,E,10,2012,22041011,AG,18198.0,872.0,872.0,2012-10-01,"Champagne, avec AOP"
4,E,10,2012,22041011,AL,557.0,40.0,36.0,2012-10-01,"Champagne, avec AOP"


In [12]:
labels1.info();

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9563 entries, 0 to 9562
Data columns (total 2 columns):
Code_NC8    9563 non-null object
Label       9563 non-null object
dtypes: object(2)
memory usage: 149.5+ KB


In [13]:
print(labels1[labels1.Code_NC8 == '22041011'].head());

      Code_NC8                Label
2218  22041011  Champagne, avec AOP


In [14]:
print(wine_exports[wine_exports.Code_NC8 == '22041011'].head());

      Flux  Mois  Annee  Code_NC8 Country      EUROS       KG     USUP  \
18798    E    10   2012  22041011      AD   229331.0  11380.0   9384.0   
18799    E    10   2012  22041011      AE  2420711.0  94908.0  83784.0   
18800    E    10   2012  22041011      AF    13950.0   1380.0   1800.0   
18801    E    10   2012  22041011      AG    18198.0    872.0    872.0   
18802    E    10   2012  22041011      AL      557.0     40.0     36.0   

           Month  
18798 2012-10-01  
18799 2012-10-01  
18800 2012-10-01  
18801 2012-10-01  
18802 2012-10-01  


In [15]:
def normalize_wine_name(name):
    return normalize_wine_name_and_variant(name)[0];

def normalize_wine_variant(name):
    return normalize_wine_name_and_variant(name)[1];




wine_exports2['wine'] =wine_exports2['Label'].apply(normalize_wine_name)
wine_exports2['variant'] =wine_exports2['Label'].apply(normalize_wine_variant)


In [16]:
wine_exports2.head()

,Flux,Mois,Annee,Code_NC8,Country,EUROS,KG,USUP,Month,Label,wine,variant
0,E,10,2012,22041011,AD,229331.0,11380.0,9384.0,2012-10-01,"Champagne, avec AOP",CHAMPAGNE,MOUSSEUX
1,E,10,2012,22041011,AE,2420711.0,94908.0,83784.0,2012-10-01,"Champagne, avec AOP",CHAMPAGNE,MOUSSEUX
2,E,10,2012,22041011,AF,13950.0,1380.0,1800.0,2012-10-01,"Champagne, avec AOP",CHAMPAGNE,MOUSSEUX
3,E,10,2012,22041011,AG,18198.0,872.0,872.0,2012-10-01,"Champagne, avec AOP",CHAMPAGNE,MOUSSEUX
4,E,10,2012,22041011,AL,557.0,40.0,36.0,2012-10-01,"Champagne, avec AOP",CHAMPAGNE,MOUSSEUX


In [17]:
%matplotlib inline
wine_exports2['wine'].value_counts()

EUROPE        48290
FRANCE        23011
OTHER         14084
BORDEAUX      13867
BOURGOGNE     10989
CHAMPAGNE     10606
LOIRE          9173
RHÔNE          6460
LANGUEDOC      4603
BEAUJOLAIS     4381
ALSACE         3726
Name: wine, dtype: int64

In [18]:
wine_exports2[wine_exports2.Country == 'IE']['variant'].value_counts()

ROUGE       1453
BLANC        250
MOUSSEUX     162
Name: variant, dtype: int64

In [19]:
wine_exports2['Label'].value_counts()[0:5]

Champagne, avec AOP                                                                                                                                                                                  6669
Vins de Bordeaux, en récipients d'une contenance <= 2 l, ayant un titre alcoométrique acquis <= 15% vol, avec AOP (à l'excl. des vins mousseux, des vins pétillants et des vins blancs)              6458
Vins produits dans l'UE, en récipients d'une contenance <= 2 l, ayant un titre alcoométrique acquis <= 15% vol, avec IGP (à l'excl. des vins mousseux, des vins pétillants et des vins blancs)       6054
Vins blancs produits dans l'UE, en récipients d'une contenance <= 2 l, ayant un titre alcoométrique acquis <= 15% vol, avec IGP (à l'excl. des vins mousseux et des vins pétillants)                 5329
Vins de la Vallée du Rhône, en récipients d'une contenance <= 2 l, ayant un titre alcoométrique acquis <= 15% vol, avec AOP (à l'excl. des vins mousseux, des vins pétillants et des vins blancs

In [20]:
wine_exports3 = wine_exports2[['Month' , 'wine' , 'variant', 'Country', 'EUROS', 'KG']]
wine_exports3['composite_key'] = wine_exports3['wine'] + "_" + wine_exports3['variant'] + "_" +wine_exports3['Country'];

/usr/local/lib/python3.5/dist-packages/ipykernel/__main__.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  from ipykernel import kernelapp as app


In [21]:
wine_exports3.sample(32)

,Month,wine,variant,Country,EUROS,KG,composite_key
119558,2016-06-01,EUROPE,ROUGE,CW,7729.0,2070.0,EUROPE_ROUGE_CW
69668,2013-09-01,EUROPE,ROUGE,MT,2088.0,1080.0,EUROPE_ROUGE_MT
126777,2016-04-01,BOURGOGNE,ROUGE,DK,3250.0,152.0,BOURGOGNE_ROUGE_DK
36680,2012-04-01,RHÔNE,ROUGE,GN,480.0,180.0,RHÔNE_ROUGE_GN
128444,2016-03-01,RHÔNE,ROUGE,QA,8616.0,1602.0,RHÔNE_ROUGE_QA
127222,2016-03-01,CHAMPAGNE,MOUSSEUX,RW,6248.0,432.0,CHAMPAGNE_MOUSSEUX_RW
52169,2013-06-01,EUROPE,ROUGE,IQ,8902.0,6300.0,EUROPE_ROUGE_IQ
135785,2016-01-01,BOURGOGNE,BLANC,US,1063.0,156.0,BOURGOGNE_BLANC_US
143955,2015-10-01,BOURGOGNE,ROUGE,US,7989504.0,363831.0,BOURGOGNE_ROUGE_US
66490,2012-09-01,BORDEAUX,BLANC,ES,4688.0,2664.0,BORDEAUX_BLANC_ES


In [22]:
wine_exports3.describe()

,EUROS,KG
count,1.491900e+05,1.491900e+05
mean,2.207440e+05,4.172200e+04
std,1.575743e+06,2.111715e+05
min,1.000000e+00,0.000000e+00
25%,1.239000e+03,2.250000e+02
50%,6.561000e+03,1.260000e+03
75%,4.235675e+04,9.273750e+03
max,8.537028e+07,6.678057e+06


In [23]:
wexp4 = wine_exports3.reset_index()
#wexp5 = wexp4.set_index(['Month' , 'wine', 'variant' , 'Country']).unstack('variant')
#wexp4[wexp4.Country == 'MR'].sample(100)
#wexp5.sample(100)
df_euros = pd.pivot_table(wexp4, values="EUROS", index=["Month"], columns=["composite_key"], fill_value=0) 
df_kgs = pd.pivot_table(wexp4, values="KG", index=["Month"], columns=["composite_key"], fill_value=0) 
df_euros.head()


composite_key,ALSACE_BLANC_AD,ALSACE_BLANC_AE,ALSACE_BLANC_AF,ALSACE_BLANC_AG,ALSACE_BLANC_AI,ALSACE_BLANC_AL,ALSACE_BLANC_AM,ALSACE_BLANC_AN,ALSACE_BLANC_AO,ALSACE_BLANC_AR,...,RHÔNE_ROUGE_VN,RHÔNE_ROUGE_VU,RHÔNE_ROUGE_WF,RHÔNE_ROUGE_XK,RHÔNE_ROUGE_XS,RHÔNE_ROUGE_YE,RHÔNE_ROUGE_YT,RHÔNE_ROUGE_ZA,RHÔNE_ROUGE_ZM,RHÔNE_ROUGE_ZW
Month,,,,,,,,,,,,,,,,,,,,,
2012-01-01,435,4887,0,0,0,0,0,2482,0,0,...,4703.0,3089,43,0,2949,0,563.0,23841,0,0
2012-02-01,0,41032,0,0,0,0,0,1911,0,0,...,18386.0,608,509,0,11798,0,0.0,12018,0,0
2012-03-01,257,45116,0,0,1660,0,0,564,0,0,...,2083.0,0,126,0,358,0,0.0,16223,4661,0
2012-04-01,463,3535,0,0,119,0,0,2886,0,0,...,11100.0,400,52,0,1647,0,0.0,20745,0,0
2012-05-01,103,55374,0,0,0,0,3300,0,0,0,...,9847.0,418,0,0,0,0,1306.5,4992,0,0


In [24]:
# wine_exports3.reset_index()

In [25]:
#df.columns[0:5]

In [26]:
df_euros.info() , df_kgs.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50 entries, 2012-01-01 to 2016-09-01
Columns: 2872 entries, ALSACE_BLANC_AD to RHÔNE_ROUGE_ZW
dtypes: float64(1009), int64(1863)
memory usage: 1.1 MB
<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 50 entries, 2012-01-01 to 2016-09-01
Columns: 2872 entries, ALSACE_BLANC_AD to RHÔNE_ROUGE_ZW
dtypes: float64(1163), int64(1709)
memory usage: 1.1 MB


(None, None)

In [27]:
df_euros.to_csv('French_Wine_Export_in_Euros.csv');
df_kgs.to_csv('French_Wine_Export_in_Kgs.csv');


In [28]:
# try to reduce number of countries ... for demo
# get the export value by couyntry
country_values = pd.DataFrame(wexp4[['Country' , 'EUROS']].groupby('Country')['EUROS'].sum())

In [29]:
country_values.head()

,EUROS
Country,
AD,9309583.0
AE,336817650.0
AF,185800.0
AG,4276528.0
AI,280473.0


In [30]:
country_values_10_best = country_values.sort_values(by='EUROS', ascending=False).head(10)

In [31]:
country_values_10_best.head(10)

,EUROS
Country,
GB,5.118784e+09
US,4.733608e+09
DE,3.088179e+09
BE,2.278347e+09
CN,2.103864e+09
JP,1.942364e+09
CH,1.417786e+09
HK,1.381578e+09
NL,1.380476e+09


In [32]:
Best_Customers = list(country_values_10_best.index)

In [33]:
Best_Customers

['GB', 'US', 'DE', 'BE', 'CN', 'JP', 'CH', 'HK', 'NL', 'CA']

In [34]:

def reduce_countries(cntry_name):
    if(cntry_name in Best_Customers):
        return cntry_name;
    return "OTHER";

wexp4['Country_1'] = wexp4['Country'].apply(reduce_countries);


In [35]:
wexp4.head()

,index,Month,wine,variant,Country,EUROS,KG,composite_key,Country_1
0,0,2012-10-01,CHAMPAGNE,MOUSSEUX,AD,229331.0,11380.0,CHAMPAGNE_MOUSSEUX_AD,OTHER
1,1,2012-10-01,CHAMPAGNE,MOUSSEUX,AE,2420711.0,94908.0,CHAMPAGNE_MOUSSEUX_AE,OTHER
2,2,2012-10-01,CHAMPAGNE,MOUSSEUX,AF,13950.0,1380.0,CHAMPAGNE_MOUSSEUX_AF,OTHER
3,3,2012-10-01,CHAMPAGNE,MOUSSEUX,AG,18198.0,872.0,CHAMPAGNE_MOUSSEUX_AG,OTHER
4,4,2012-10-01,CHAMPAGNE,MOUSSEUX,AL,557.0,40.0,CHAMPAGNE_MOUSSEUX_AL,OTHER


In [36]:
wexp4['composite_key_1'] = wexp4['wine'] + "_" + wexp4['variant'] + "_" + wexp4['Country_1'];

In [37]:
df_euros_1 = pd.pivot_table(wexp4, values="EUROS", index=["Month"], columns=["composite_key_1"], fill_value=0) 
df_kgs_1 = pd.pivot_table(wexp4, values="KG", index=["Month"], columns=["composite_key_1"], fill_value=0) 
df_euros_1.head()

composite_key_1,ALSACE_BLANC_BE,ALSACE_BLANC_CA,ALSACE_BLANC_CH,ALSACE_BLANC_CN,ALSACE_BLANC_DE,ALSACE_BLANC_GB,ALSACE_BLANC_HK,ALSACE_BLANC_JP,ALSACE_BLANC_NL,ALSACE_BLANC_OTHER,...,RHÔNE_ROUGE_CA,RHÔNE_ROUGE_CH,RHÔNE_ROUGE_CN,RHÔNE_ROUGE_DE,RHÔNE_ROUGE_GB,RHÔNE_ROUGE_HK,RHÔNE_ROUGE_JP,RHÔNE_ROUGE_NL,RHÔNE_ROUGE_OTHER,RHÔNE_ROUGE_US
Month,,,,,,,,,,,,,,,,,,,,,
2012-01-01,1131852,571940,257116,140013,357516,344274,51043,329883,624695,32232.388889,...,800628,650236,1127393,783952,1513332,335710,916965,314654,35418,1260510
2012-02-01,857289,585556,187917,89805,505896,424953,80220,239242,899144,26432.432836,...,795356,680567,188827,868576,1277664,197072,318522,516140,36604,2136484
2012-03-01,1438501,834142,679281,103577,583887,352675,102980,394945,1093638,38946.937500,...,873858,1318538,856382,627236,2343448,132200,605914,410308,43197,3264810
2012-04-01,986602,866221,523202,203456,515903,403517,52773,260953,874217,34391.476923,...,1004028,919493,460815,829432,1861188,128164,472522,290407,45396,2631466
2012-05-01,991668,1061702,611369,80327,767522,627232,78115,680323,994697,45811.645161,...,2098300,632778,239200,934532,2215922,202210,337894,284868,41469,2945175


In [38]:
df_euros_1.to_csv('French_Wine_Export_in_Euros_Some_Countries.csv');
df_kgs_1.to_csv('French_Wine_Export_in_Kgs_Some_Countries.csv');